# PyTorch Deep Learning Tutorials

This notebook demonstrates basic PyTorch usage and simple neural network examples. Run the cell below if you need to install PyTorch. In Google Colab it is already available.


In [ ]:
!pip install torch torchvision

In [ ]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## 0. Tensor basics

In [ ]:
a = torch.tensor([[1.,2.],[3.,4.]])
b = torch.rand(2,2)
print('a:', a)
print('b:', b)
print('a + b:', a + b)
print('a @ b:', a @ b)
x = torch.tensor([2.0], requires_grad=True)
y = x**2 + 3*x + 1
y.backward()
print('dy/dx at x=2:', x.grad.item())

## 1. Train a simple MLP

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
full_train = datasets.MNIST(root='data', train=True, download=True, transform=transform)
train_size = int(0.8 * len(full_train))
val_size = len(full_train) - train_size
train_ds, val_ds = torch.utils.data.random_split(full_train, [train_size, val_size])
test_ds = datasets.MNIST(root='data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64)
test_loader = DataLoader(test_ds, batch_size=64)
model = nn.Sequential(nn.Flatten(), nn.Linear(28*28, 128), nn.ReLU(), nn.Linear(128,10)).to(device)
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
train_losses, val_losses = [], []
for epoch in range(3):
    model.train()
    epoch_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        opt.zero_grad()
        out = model(imgs)
        loss = loss_fn(out, labels)
        loss.backward()
        opt.step()
        epoch_loss += loss.item()
    train_losses.append(epoch_loss / len(train_loader))
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            out = model(imgs)
            val_loss += loss_fn(out, labels).item()
    val_losses.append(val_loss / len(val_loader))
    print(f'Epoch {epoch+1}: train_loss={train_losses[-1]:.4f}, val_loss={val_losses[-1]:.4f}')

## 2. Evaluate and visualize

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_losses, label='train')
plt.plot(val_losses, label='val')
plt.xlabel('epoch'); plt.ylabel('loss'); plt.legend(); plt.show()
correct = 0
model.eval()
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        pred = model(imgs).argmax(dim=1)
        correct += (pred == labels).sum().item()
acc = correct / len(test_ds)
print('Test accuracy:', acc)

This short notebook gives a taste of working with PyTorch. Feel free to modify the code cells and experiment further!